In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from scipy import signal
import pywt
import pickle
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# 设置随机种子以确保结果可重现
np.random.seed(42)

# 创建figure目录（如果不存在）
os.makedirs('figure', exist_ok=True)

# 加载训练和测试数据
train_data = pd.read_excel('/data/jinming/ee_prediction/data/train.xlsx')
test_data = pd.read_excel('/data/jinming/ee_prediction/data/test.xlsx')

# 提取功率数据（第一列是实际功率 - 已归一化）
train_power = train_data.iloc[:, 0].values
test_power = test_data.iloc[:, 0].values

# 数据已经归一化，无需再次归一化
train_power_scaled = train_power
test_power_scaled = test_power

print(f"训练数据形状: {train_power_scaled.shape}")
print(f"测试数据形状: {test_power_scaled.shape}")

# CR指标计算函数
def calculate_CR(PM, PP):
    """
    计算CR指标（准确度指标）
    
    参数:
    PM: 实际值数组
    PP: 预测值数组
    
    返回:
    CR: CR指标值（百分比）
    """
    N = len(PM)
    Ri = np.zeros(N)
    for i in range(N):
        if PM[i] > 0.2:
            Ri[i] = (PM[i] - PP[i]) / PM[i]
        else:
            Ri[i] = (PM[i] - PP[i]) / 0.2
    
    rms_error = np.sqrt(np.mean(Ri**2))
    CR = (1 - rms_error) * 100
    return CR

# 准备训练数据：为3个不同预测时长创建数据集
def create_dataset_for_horizon(data, input_size=96, predict_horizon=1, step=1):
    """
    为特定预测时长创建训练数据集
    
    参数:
    data: 时间序列数据
    input_size: 输入窗口大小
    predict_horizon: 预测时长（1=15分钟, 4=1小时, 16=4小时）
    step: 滑动窗口步长
    
    返回:
    X: 输入序列，形状 (样本数量, 输入长度)
    y: 输出值，形状 (样本数量, 1)
    """
    X, y = [], []
    for i in range(0, len(data) - input_size - predict_horizon, step):
        X.append(data[i:i+input_size])
        y.append(data[i+input_size+predict_horizon-1])
    return np.array(X), np.array(y).reshape(-1, 1)

# 分割训练和验证集
def split_train_val(X, y, train_ratio=0.8):
    train_size = int(len(X) * train_ratio)
    X_train = X[:train_size]
    y_train = y[:train_size]
    X_val = X[train_size:]
    y_val = y[train_size:]
    return X_train, y_train, X_val, y_val

# 带早停机制的BP神经网络类
class BPNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size=1, learning_rate=0.01, use_adam=True):
        """
        初始化BP神经网络
        
        参数:
        input_size: 输入层节点数
        hidden_size: 隐藏层节点数
        output_size: 输出层节点数（默认为1）
        learning_rate: 学习率
        use_adam: 是否使用Adam优化器
        """
        # 网络结构参数
        self.input_size = input_size    # 输入层节点数
        self.hidden_size = hidden_size  # 隐藏层节点数
        self.output_size = output_size  # 输出层节点数
        self.learning_rate = learning_rate  # 学习率
        self.use_adam = use_adam  # 是否使用Adam优化器
        
        # 初始化网络权重和偏置
        # Xavier初始化，用于更稳定的反向传播
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(1/input_size)  # 输入层到隐藏层权重
        self.b1 = np.zeros((1, hidden_size))  # 隐藏层偏置
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(1/hidden_size)  # 隐藏层到输出层权重
        self.b2 = np.zeros((1, output_size))  # 输出层偏置
        
        # Adam优化器参数
        if use_adam:
            # 动量参数
            self.beta1 = 0.9
            self.beta2 = 0.999
            self.epsilon = 1e-8
            
            # 初始化梯度动量和平方梯度动量
            self.m_W1 = np.zeros_like(self.W1)
            self.m_b1 = np.zeros_like(self.b1)
            self.m_W2 = np.zeros_like(self.W2)
            self.m_b2 = np.zeros_like(self.b2)
            
            self.v_W1 = np.zeros_like(self.W1)
            self.v_b1 = np.zeros_like(self.b1)
            self.v_W2 = np.zeros_like(self.W2)
            self.v_b2 = np.zeros_like(self.b2)
            
            # 时间步计数器
            self.t = 0
    
    def sigmoid(self, x):
        """
        Sigmoid激活函数: f(x) = 1 / (1 + e^(-x))
        将输入映射到(0,1)范围
        """
        # 防止数值溢出
        x = np.clip(x, -500, 500)
        return 1.0 / (1.0 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        """
        Sigmoid函数导数: f'(x) = f(x) * (1 - f(x))
        用于反向传播梯度计算
        """
        return x * (1.0 - x)
    
    def forward(self, X):
        """
        前向传播算法
        
        参数:
        X: 输入数据，形状 (样本数量, 输入维度)
        
        返回:
        输出层输出
        """
        # 第一层：输入层到隐藏层
        self.z1 = np.dot(X, self.W1) + self.b1  # 隐藏层加权输入
        self.a1 = self.sigmoid(self.z1)         # 隐藏层激活输出
        
        # 第二层：隐藏层到输出层
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # 输出层加权输入
        self.a2 = self.sigmoid(self.z2)              # 输出层激活输出
        
        return self.a2
    
    def backward(self, X, y, output):
        """
        反向传播算法
        
        参数:
        X: 输入数据，形状 (样本数量, 输入维度)
        y: 目标输出，形状 (样本数量, 输出维度)
        output: 前向传播输出
        
        返回:
        当前批次误差
        """
        # 计算输出层误差
        output_error = y - output  # 输出误差（目标值减去预测值）
        output_delta = output_error * self.sigmoid_derivative(output)  # 输出层梯度
        
        # 计算隐藏层误差
        hidden_error = np.dot(output_delta, self.W2.T)  # 通过链式法则计算隐藏层误差
        hidden_delta = hidden_error * self.sigmoid_derivative(self.a1)  # 隐藏层梯度
        
        # 更新权重和偏置
        m = X.shape[0]  # 样本数量
        
        # 计算梯度
        dW2 = np.dot(self.a1.T, output_delta) / m
        db2 = np.sum(output_delta, axis=0, keepdims=True) / m
        dW1 = np.dot(X.T, hidden_delta) / m
        db1 = np.sum(hidden_delta, axis=0, keepdims=True) / m
        
        if self.use_adam:
            # Adam优化器更新
            self.t += 1
            
            # 更新动量
            self.m_W2 = self.beta1 * self.m_W2 + (1 - self.beta1) * dW2
            self.m_b2 = self.beta1 * self.m_b2 + (1 - self.beta1) * db2
            self.m_W1 = self.beta1 * self.m_W1 + (1 - self.beta1) * dW1
            self.m_b1 = self.beta1 * self.m_b1 + (1 - self.beta1) * db1
            
            # 更新平方梯度动量
            self.v_W2 = self.beta2 * self.v_W2 + (1 - self.beta2) * (dW2 ** 2)
            self.v_b2 = self.beta2 * self.v_b2 + (1 - self.beta2) * (db2 ** 2)
            self.v_W1 = self.beta2 * self.v_W1 + (1 - self.beta2) * (dW1 ** 2)
            self.v_b1 = self.beta2 * self.v_b1 + (1 - self.beta2) * (db1 ** 2)
            
            # 偏置校正
            m_W2_corrected = self.m_W2 / (1 - self.beta1 ** self.t)
            m_b2_corrected = self.m_b2 / (1 - self.beta1 ** self.t)
            m_W1_corrected = self.m_W1 / (1 - self.beta1 ** self.t)
            m_b1_corrected = self.m_b1 / (1 - self.beta1 ** self.t)
            
            v_W2_corrected = self.v_W2 / (1 - self.beta2 ** self.t)
            v_b2_corrected = self.v_b2 / (1 - self.beta2 ** self.t)
            v_W1_corrected = self.v_W1 / (1 - self.beta2 ** self.t)
            v_b1_corrected = self.v_b1 / (1 - self.beta2 ** self.t)
            
            # 使用Adam更新规则更新权重
            self.W2 += self.learning_rate * m_W2_corrected / (np.sqrt(v_W2_corrected) + self.epsilon)
            self.b2 += self.learning_rate * m_b2_corrected / (np.sqrt(v_b2_corrected) + self.epsilon)
            self.W1 += self.learning_rate * m_W1_corrected / (np.sqrt(v_W1_corrected) + self.epsilon)
            self.b1 += self.learning_rate * m_b1_corrected / (np.sqrt(v_b1_corrected) + self.epsilon)
        else:
            # 使用传统梯度下降更新权重
            self.W2 += self.learning_rate * dW2
            self.b2 += self.learning_rate * db2
            self.W1 += self.learning_rate * dW1
            self.b1 += self.learning_rate * db1
        
        # 计算总误差（均方误差）
        total_error = np.mean(np.sum(np.square(output_error), axis=1))
        return total_error
    
    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=100, batch_size=32, 
              shuffle=False, verbose=True, early_stopping=True, patience=200, min_delta=1e-6):
        """
        训练神经网络（带早停机制）
        
        参数:
        X_train: 训练输入数据
        y_train: 训练目标数据
        X_val: 验证集输入数据（可选）
        y_val: 验证集目标数据（可选）
        epochs: 训练轮数
        batch_size: 每次更新的批次大小
        shuffle: 是否打乱数据
        verbose: 是否打印训练进度
        early_stopping: 是否启用早停机制
        patience: 早停容忍轮数
        min_delta: 最小改进阈值
        
        返回:
        train_losses: 训练损失历史
        val_losses: 验证损失历史
        val_cr_scores: 验证CR指标历史
        """
        train_losses = []  # 存储每轮训练损失
        val_losses = []    # 存储每轮验证损失
        val_cr_scores = [] # 存储每轮验证CR指标
        n_samples = X_train.shape[0]  # 总样本数
        
        # 早停相关变量
        best_cr_score = -np.inf  # 最佳CR指标（越高越好）
        patience_counter = 0     # 耐心计数器
        best_weights = None      # 最佳权重
        
        for epoch in range(epochs):
            if shuffle:
                # 打乱数据以获得更稳定和鲁棒的训练
                indices = np.random.permutation(n_samples)
                X_shuffled = X_train[indices]
                y_shuffled = y_train[indices]
            else:
                X_shuffled = X_train
                y_shuffled = y_train
            
            epoch_loss = 0
            # 小批量梯度下降，每次用batch_size个样本更新权重
            for i in range(0, n_samples, batch_size):
                end = min(i + batch_size, n_samples)
                batch_X = X_shuffled[i:end]
                batch_y = y_shuffled[i:end]
                
                # 前向传播
                output = self.forward(batch_X)
                
                # 反向传播和权重更新
                batch_loss = self.backward(batch_X, batch_y, output)
                epoch_loss += batch_loss * (end - i) / n_samples
            
            train_losses.append(epoch_loss)
            
            # 在验证集上评估（如果提供）
            if X_val is not None and y_val is not None:
                val_output = self.forward(X_val)
                val_loss = np.mean(np.sum(np.square(y_val - val_output), axis=1))
                val_losses.append(val_loss)
                
                # 计算CR指标
                val_predictions = val_output.flatten()
                val_actual = y_val.flatten()
                val_cr_score = calculate_CR(val_actual, val_predictions)
                val_cr_scores.append(val_cr_score)
                
                # 早停检查
                if early_stopping:
                    if val_cr_score > best_cr_score + min_delta:
                        best_cr_score = val_cr_score
                        patience_counter = 0
                        # 保存最佳权重
                        best_weights = {
                            'W1': self.W1.copy(),
                            'b1': self.b1.copy(), 
                            'W2': self.W2.copy(),
                            'b2': self.b2.copy()
                        }
                    else:
                        patience_counter += 1
                    
                    # 如果超过耐心值，停止训练
                    if patience_counter >= patience:
                        if verbose:
                            print(f"\n早停触发！在第 {epoch+1} 轮停止训练")
                            print(f"最佳CR指标: {best_cr_score:.4f}%")
                        # 恢复最佳权重
                        if best_weights:
                            self.W1 = best_weights['W1']
                            self.b1 = best_weights['b1']
                            self.W2 = best_weights['W2']
                            self.b2 = best_weights['b2']
                        break
                
                # 打印训练进度
                if verbose and (epoch % 200 == 0 or epoch == epochs - 1):
                    print(f"轮次 {epoch+1}/{epochs}, 训练损失: {epoch_loss:.6f}, 验证损失: {val_loss:.6f}, CR指标: {val_cr_score:.4f}%")
            else:
                # 打印训练进度
                if verbose and (epoch % 200 == 0 or epoch == epochs - 1):
                    print(f"轮次 {epoch+1}/{epochs}, 训练损失: {epoch_loss:.6f}")
        
        return train_losses, val_losses if X_val is not None else train_losses, val_cr_scores if X_val is not None else []
    
    def predict(self, X):
        """
        预测函数
        
        参数:
        X: 输入数据
        
        返回:
        预测结果
        """
        return self.forward(X)

# 1. 自相关分析方法选择输入窗口大小
def analyze_window_size_with_autocorrelation(data, max_lag=100):
    """
    使用自相关和偏自相关分析确定合适的输入窗口大小
    
    参数:
    data: 时间序列数据
    max_lag: 最大滞后阶数
    
    返回:
    suggested_window: 建议的输入窗口大小
    """
    plt.figure(figsize=(16, 8))
    
    # 自相关分析
    plt.subplot(211)
    plot_acf(data, lags=max_lag, alpha=0.05, title='自相关函数 (ACF)')
    
    # 偏自相关分析
    plt.subplot(212)
    plot_pacf(data, lags=max_lag, alpha=0.05, method='ywm', title='偏自相关函数 (PACF)')
    
    plt.tight_layout()
    plt.savefig('figure/acf_pacf_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 计算自相关系数
    acf_values = sm.tsa.acf(data, nlags=max_lag, fft=True)
    
    # 找出自相关系数首次降到0.2以下的滞后阶数
    for i, acf_val in enumerate(acf_values):
        if i > 0 and abs(acf_val) < 0.2:
            return i
    
    # 如果所有自相关系数都较高，建议使用最大滞后阶数的一半
    return max(24, max_lag // 2)

# 2. 简化的窗口大小选择方法
def find_optimal_window_size_simple(data, window_sizes_to_test=[16, 24, 32, 48, 64, 96]):
    """
    使用简单的训练验证集划分找到最优输入窗口大小
    
    参数:
    data: 时间序列数据
    window_sizes_to_test: 要测试的窗口大小列表
    
    返回:
    best_window: 最优窗口大小
    window_scores: 各窗口大小的性能评分
    """
    best_window = 32  # 默认值
    best_score = -np.inf
    
    # 分割数据为训练集和验证集（8:2）
    train_size = int(len(data) * 0.8)
    train_data = data[:train_size]
    val_data = data[train_size:]
    
    window_scores = []
    
    print("开始测试不同窗口大小...")
    for window_size in window_sizes_to_test:
        # 创建数据集
        X_train, y_train = create_dataset_for_horizon(train_data, window_size, 1, 1)
        X_val, y_val = create_dataset_for_horizon(val_data, window_size, 1, 1)
        
        if len(X_train) == 0 or len(X_val) == 0:
            continue
        
        # 训练简化模型
        model = BPNeuralNetwork(window_size, 32, 1, 0.01, use_adam=True)
        model.train(X_train, y_train, epochs=300, batch_size=32, verbose=False)
        
        # 预测并计算CR指标
        y_pred = model.predict(X_val)
        cr_score = calculate_CR(y_val.flatten(), y_pred.flatten())
        mse = mean_squared_error(y_val, y_pred)
        
        window_scores.append((window_size, mse, cr_score))
        print(f"窗口大小 {window_size}: MSE = {mse:.6f}, CR = {cr_score:.2f}%")
        
        if cr_score > best_score:
            best_score = cr_score
            best_window = window_size
    
    # 绘制窗口大小vs.性能指标曲线
    plt.figure(figsize=(12, 8))
    
    plt.subplot(211)
    plt.plot([w for w, m, c in window_scores], [m for w, m, c in window_scores], 'o-')
    plt.xlabel('输入窗口大小')
    plt.ylabel('MSE')
    plt.title('不同输入窗口大小的MSE对比')
    plt.grid(True)
    
    plt.subplot(212)
    plt.plot([w for w, m, c in window_scores], [c for w, m, c in window_scores], 'o-')
    plt.axvline(x=best_window, color='r', linestyle='--', label=f'最佳窗口: {best_window}')
    plt.xlabel('输入窗口大小')
    plt.ylabel('CR指标 (%)')
    plt.title('不同输入窗口大小的CR指标对比')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('figure/window_size_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return best_window, window_scores

# 3. 添加时间特征
def add_time_features(data, timestamps=None):
    """
    为时间序列数据添加时间特征
    
    参数:
    data: 时间序列数据
    timestamps: 时间戳序列（如果没有提供，则假设等间隔采样，间隔15分钟）
    
    返回:
    features: 添加时间特征后的数据
    """
    n = len(data)
    
    # 如果没有提供时间戳，创建一个假设的时间序列（间隔15分钟）
    if timestamps is None:
        start_time = pd.Timestamp('2021-01-01')
        timestamps = [start_time + pd.Timedelta(minutes=15*i) for i in range(n)]
    
    # 创建DataFrame
    df = pd.DataFrame({
        'power': data,
        'timestamp': timestamps
    })
    
    # 提取时间特征
    df['hour'] = df['timestamp'].dt.hour
    df['day'] = df['timestamp'].dt.day
    df['month'] = df['timestamp'].dt.month
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['quarter'] = df['timestamp'].dt.quarter
    
    # 添加周期性特征（使用正弦和余弦变换）
    # 小时周期性 (24小时周期)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # 一周内的天周期性 (7天周期)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    # 一年中的天周期性 (365天周期)
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    
    # 将数据转换为numpy数组
    feature_columns = ['power', 'hour_sin', 'hour_cos', 'day_of_week_sin', 
                      'day_of_week_cos', 'day_of_year_sin', 'day_of_year_cos']
    features = df[feature_columns].values
    
    return features

# 4. 差分处理
def differencing(data, order=1):
    """
    对时间序列进行差分处理
    
    参数:
    data: 时间序列数据
    order: 差分阶数
    
    返回:
    diff_data: 差分后的数据
    """
    diff_data = data.copy()
    for _ in range(order):
        diff_data = np.diff(diff_data, prepend=diff_data[0])
    return diff_data

# 5. 小波变换处理
def wavelet_features(data, wavelet='db4', level=3):
    """
    使用小波分解提取多尺度特征
    
    参数:
    data: 时间序列数据
    wavelet: 小波类型
    level: 分解级别
    
    返回:
    features: 小波特征
    """
    # 确保数据长度是2的幂次方
    n = len(data)
    pad_size = int(2**np.ceil(np.log2(n))) - n
    padded_data = np.pad(data, (0, pad_size), 'constant', constant_values=(0, 0))
    
    # 进行小波分解
    coeffs = pywt.wavedec(padded_data, wavelet, level=level)
    
    # 使用小波系数作为特征
    features = np.concatenate([coeffs[0]] + [c for c in coeffs[1:]], axis=0)[:n]
    
    return features.reshape(-1, 1)

# 对训练数据进行自相关分析
print("\n开始使用自相关分析方法确定输入窗口大小...")
suggest_window_size = analyze_window_size_with_autocorrelation(train_power_scaled, max_lag=100)
print(f"自相关分析建议的输入窗口大小: {suggest_window_size}")

# 使用简化方法测试不同窗口大小
print("\n开始测试不同窗口大小的性能...")
window_sizes_to_test = [16, 24, 32, 48, 64, 96]
best_window, window_scores = find_optimal_window_size_simple(train_power_scaled, window_sizes_to_test)
print(f"\n最佳输入窗口大小: {best_window}")

# 最终确定的输入窗口大小
final_input_size = best_window
print(f"\n最终确定的输入窗口大小: {final_input_size}")

# 分析最终选择的窗口大小对应的自相关性
plt.figure(figsize=(10, 6))
plot_acf(train_power_scaled, lags=final_input_size, alpha=0.05, title=f'功率数据自相关函数 (滞后阶数 = {final_input_size})')
plt.grid(True)
plt.savefig('figure/final_window_acf.png', dpi=300, bbox_inches='tight')
plt.close()

# 可视化时间序列数据的周期性特征
plt.figure(figsize=(14, 8))

# 绘制原始功率数据
plt.subplot(311)
plt.plot(train_power_scaled[:1000])
plt.title('风电功率时间序列 (前1000个样本)')
plt.ylabel('归一化功率')
plt.grid(True)

# 绘制功率数据的频谱
plt.subplot(312)
f, Pxx = signal.periodogram(train_power_scaled, fs=96)  # 假设每天96个样本点
plt.semilogy(f[:len(f)//10], Pxx[:len(Pxx)//10])
plt.title('功率数据频谱')
plt.xlabel('频率 (周期/天)')
plt.ylabel('功率谱密度')
plt.grid(True)

# 绘制差分后的数据
plt.subplot(313)
diff_data = differencing(train_power_scaled)
plt.plot(diff_data[:1000])
plt.title('一阶差分后的风电功率时间序列')
plt.xlabel('时间点')
plt.ylabel('差分值')
plt.grid(True)

plt.tight_layout()
plt.savefig('figure/time_series_analysis.png', dpi=300, bbox_inches='tight')
plt.close()

print("\n时间序列分析完成，结果已保存到figure文件夹")